In [1]:
from rocketpy import Environment, Fluid, CylindricalTank, MassFlowRateBasedTank, HybridMotor, Rocket, Flight
import numpy as np

In [2]:
#Definir environment
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)
date_info = (tomorrow.year, tomorrow.month, tomorrow.day, 12)  # Hour given in UTC time
print("Tomorrow's date:", date_info)


env = Environment(date=date_info, latitude=39.389700, longitude=-8.288964, elevation=160)
env.set_atmospheric_model(type="Windy", file="ICONEU")

Tomorrow's date: (2025, 7, 22, 12)


In [3]:
# Define the fluids
oxidizer_liq = Fluid(name="N2O_l", density=785.1)
oxidizer_gas = Fluid(name="N2_g", density=157.99) 


# Define tank geometry
cylindrical_geometry = CylindricalTank(radius=0.0815, height=0.550, spherical_caps=True) #0.075



# Define tank
oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=cylindrical_geometry,
    flux_time=4.2, 
    initial_liquid_mass=6.3,  

    initial_gas_mass=0, #0.640
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=1.5,   
    
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)



#Define motor

example_hybrid = HybridMotor(
    
    coordinate_system_orientation="nozzle_to_combustion_chamber",
    thrust_source="curves/Thrust.csv",    #"Novo_thrust_estimado.txt",  
    dry_mass=21.5,                       
    center_of_dry_mass_position=0.826, 
    dry_inertia=(6.635, 6.635, 0.06135), 
    
    
    grain_number=1,   
    grain_separation=0,     
    grain_outer_radius=0.0575,          
    grain_initial_inner_radius=0.020,   
    grain_initial_height=0.35,      
    grain_density=888,            
    grains_center_of_mass_position=0.175+0.270, #CG referencial do grain + Distancia ao nozzle  
    
    
    nozzle_radius=0.048074,
    nozzle_position=0, 
    throat_radius=0.030, 
   
)



example_hybrid.add_tank(
  tank = oxidizer_tank, position = 1.07+(0.550/2)
)


#Estrutura do rocket

INVICTUS = Rocket(
    
    coordinate_system_orientation = "nose_to_tail", 
    center_of_mass_without_motor=2.09, 
    radius=0.0815,   
    mass=20.0,
    inertia=(31.99, 31.99, 0.2049),  
    
    power_off_drag="curves/CURVA_DRAG.csv", 
    power_on_drag="curves/CURVA_DRAG.csv", 


)

INVICTUS.add_motor(example_hybrid, position=3.75+0.121) #posição em relação ao CG do motor completo (inclui tanque)

#Nose Cone

NoseCone = INVICTUS.add_nose(
    length=0.385, kind="von karman", position=0   #kind="parabolic"
)
 
    
#Fins
fin_set = INVICTUS.add_trapezoidal_fins(
    n=4,
    root_chord=0.28,  
    tip_chord=0.14,  
    span=0.160,  
    position=3.6, 
    airfoil=None,
    sweep_length=0.07, #isto para ter a forma trapezoidal
)

#Tail

Tail = INVICTUS.add_tail(
    top_radius=0.0815, bottom_radius=0.058, length=0.121, position=3.75 
)

#Rail Buttons
rail_buttons = INVICTUS.set_rail_buttons(
    upper_button_position=1.306,
    lower_button_position=2.8635,
    angular_position=45,
)

#Paraquedas
Drogue = INVICTUS.add_parachute(
    "Drogue",
    cd_s=0.7354,
    trigger="apogee",
    sampling_rate=10,
    lag=1, 
)

Main = INVICTUS.add_parachute(
    "Main",
    cd_s=13.8991,
    trigger=450,
    sampling_rate=10,
    lag=1,  
)


#Rail

test_flight = Flight(
    rocket=INVICTUS, 
    environment=env, 
    rail_length=12, 
    inclination=84, #90 #84
    heading=0, #133
)

In [ ]:
example_hybrid.all_info()

In [ ]:
INVICTUS.all_info()

In [ ]:
test_flight.all_info()

In [ ]:
###### ANÁLISE VELOCIDADE TERMINAL 

drogue_t = test_flight.parachute_events[0][0]
main_t = test_flight.parachute_events[1][0]
print(f"Rocket Dry Mass: {INVICTUS.dry_mass}kg")
print(f"Drogue opening: {drogue_t}s")
print(f"Main Opening: {main_t}s")
test_flight.vz()

print(f"Main terminal velocity: {test_flight.vz(140)}")
print(f"Drogue terminal velocity: {test_flight.vz(110)}")

test_flight.acceleration.plot(113, 113.02)
print(test_flight.ax(113.01))
print(test_flight.ay(113.01))
print(test_flight.az(113.01))
